In [13]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [114]:
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

search = DuckDuckGoSearchRun()

search.invoke("Obama's first name?")

"The White House, official residence of the president of the United States, in July 2008. The president of the United States is the head of state and head of government of the United States, [1] indirectly elected to a four-year term via the Electoral College. [2] Under the U.S. Constitution, the officeholder leads the executive branch of the federal government and is the commander-in-chief of ... Barack Obama, the 44th President of the United States, broke barriers as the first African-American president and implemented significant healthcare reforms during his tenure. ... Secret Service Code Names of 11 U.S. Presidents. Barack Obama's Presidential Legacy. 10 Democrats Who Made History. Discover ... Barack Obama's first book, Dreams ... Since 1789 and the election of George Washington, America's first president, 45 individuals have served as the chief executive of the United States ... Barack Obama: Joseph Biden: Democratic: 2009-2017: Donald Trump: Mike Pence: Republican: 2017-2021: 

In [115]:
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

'성남시 북데이423 카카오판교아지트점에 대한 장소정보(전화번호,주소,오시는길안내,지도,로드뷰,홈페이지,교통정보,리뷰등) 📌 경기 성남시 분당구 판교역로 166 카카오판교아지트 1층. 📌 평일 10:00 - 21:00 / 주말 12:00 - 20:00. 존재하지 않는 스티커입니다. 카카오프렌즈 판교아지트점은 판교역 4번출구로 나오면 바로 앞 건물입니다. 건물이 카카오 사옥과 같이 있어서 1층은 식당, 카페와 카카오프렌즈샵이고 위에는 카카오 사무실이 있는 것 같았어요. 지하철과 연결되어 있어서 더욱 접근이 편리합니다. 존재하지 않는 이미지입니다. 건물 1층에 있는데 밖에서는 따로 간판같은게 없어서 살짝 헤맸어요. 통유리 건물이라 안을 들여다봐야 매장이 보이더라구요. 밤이면 잘 보일텐데 낮이라 밖에 더 밝아서 빙빙 돌아서 들어갔습니다. - 주소 : 경기도 성남시 분당구 판교역로 166 카카오 판교아지트 1층, 카카오쇼핑라이브 옆 - 카카오맵 : https://place.map.kakao.com/143299114 . 2_카카오프렌즈 판교아지트점 위치, 둘러보기. 카카오프렌즈 판교아지트점은 카카오 판교아지트 건물 1층에 있습니다. 판교카페 카카오 프렌즈아지트 디저트 맛집 플링크 날이 좋았던 지난 주말 판교에서 저녁 약속이... < 판교카페 플링크 위치 정보 신분당선.. 비서제이 상세보기 naver 블로그. 일상을 여행하는 중. 블로그 검색'

In [22]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.tools.retriever import create_retriever_tool

bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
)

<SoupStrainer name=[<TagNameMatchRule string=div pattern=None function=None present=None>] attrs=defaultdict(<class 'list'>, {'class': [<AttributeValueMatchRule string=newsct_article _article_body pattern=None function=None present=None>, <AttributeValueMatchRule string=media_end_head_title pattern=None function=None present=None>]}) string=[]>

In [6]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs


문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 

In [8]:
ext_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

In [23]:
embeddings = UpstageEmbeddings(model="embedding-query")
llm = ChatUpstage(model="solar-mini")

In [15]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [21]:
retriever.invoke("부영그룹의 정책에 대해서 알려줘")

[Document(id='b517f6ea-47b7-41d1-b310-943f6df2c792', metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은

In [78]:
retriever_tool = create_retriever_tool(
    retriever,
    name="부영_출산정책",  # 도구의 이름을 입력합니다.
    # description="use this tool to search information from the news",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
    description="부영그룹이나 저출생 정책에 관해 검색할때는 이 tool을 이용하세요",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

In [79]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]
# tools = [search]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are a helpful assistant. "
            "당신은 유용한 도우미입니다. "
            # "Make sure to use the `news_search` tool for searching information from the news. "
            "부영그룹에 대한 정보나 저출생 대책에 관한 정보를 검색할 때는 `부영_출산정책` 도구를 사용하세요. "
            "만약 뉴스에서 정보를 찾을 수 없다면, `duckduckgo_search` 도구를 사용하여 웹에서 정보를 검색하세요.",
            # "If you can't find the information from the news, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [81]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

In [82]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [83]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [100]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    # {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
    # {"input": "부영그룹의 임직원 숫자는 몇명인가요?"}
    {"input": "윤석열 대통령은 탄핵됐어?"}
    
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: duckduckgo_search
query: 윤석열 대통령 탄핵
Log: 
Invoking: `duckduckgo_search` with `{'query': '윤석열 대통령 탄핵'}`



[관찰 내용]
Observation: 4일 오전 11시 헌법재판소가 윤석열 대통령 파면 여부를 결정한다. 12·3 비상계엄 선포로부터 123일, 국회가 탄핵소추안을 가결한 지 111일 만이다 ... 헌법재판소가 4일 오전, 재판관 전원일치로 탄핵소추를 인용하면서 윤석열 전 대통령은 8년 만에, 헌정사상 두 번째 대통령 파면이라는 불명예 ... 윤석열 대통령이 위헌·위법적 비상계엄과 내란 행위로 탄핵소추안을 발의한 것이 국회에서 204표 찬성으로 가결됐다. 탄핵 사유는 국군통수권, 조약체결, 사면·감형·복권 등 대통령의 권한과 직무가 정지되고 한덕수 국무총리가 대행하게 된다. 헌법재판소가 국회의 탄핵소추 청구를 인용하면 윤 대통령은 박근혜 전 대통령에 이어 임기 중 파면되는 두 번째 대통령으로 기록된다. 윤 대통령 탄핵안은 이날 오후 4시 본회의에서 재적 의원 300명 중 300명 전원이 참석한 가운데 찬성 204표, 반대 85표, 기권 3표, 무효 8표로 가결됐다. 탄핵안이 가결되려면 재적의원의 3분의 2 (200명)가 찬성해야 하는데, 204명의 찬성으로 가결 정족수를 넘겼다. 범야권 192명이 탄핵 찬성 의사를 밝힌 점을 고려하면, '부결 당론'을 유지한 국민의힘에서 '찬성' 이탈표가 12표 나온 것으로 보인다. 기권 및 무효표 11명도 국민의힘에서 나온 것으로 추정된다. 헌법재판소의 윤석열 대통령 탄핵 심판 마지막 변론이 25일 마무리됐다. 윤 대통령이 지난해 12·3 비상계엄을 선포한 지 84일 만, 지난해 12월 14일 국회에서 윤 대통령 탄핵소추안이 가결된 지 73일 만이다. 헌법재판소는 이날 오후 2시 대심판정에서 윤 대통령 탄핵심판의 11차 변론을 열었다. 이날 변론은 증거조사, 양측 종합변론,...
[최종 답변]
윤석열 대통령은 탄핵

In [87]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "부영그룹의 출산 장려 정책에 대한 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: 부영_출산정책
query: 부영그룹 출산 장려 정책
Log: 
Invoking: `부영_출산정책` with `{'query': '부영그룹 출산 장려 정책'}`



[관찰 내용]
Observation: [앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 직원들의 세금 

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [93]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "부영그룹의 출산 장려 정책에 대한 정보를 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
부영그룹은 저출생 대책으로 직원 자녀에게 1억원씩 지원하는 정책을 시행하고 있습니다. 이 정책은 2021년 이후 태어난 직원 자녀에게 적용되며, 연년생과 쌍둥이 자녀의 경우 총 2억원을 받게 됩니다. 또한, 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.


In [98]:
response = agent_with_chat_history.stream(
    {"input": "윤석열 대통령 탄핵됐어?"},
    # {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
현재로서는 윤석열 대통령이 탄핵되었는지 여부를 알 수 없습니다. 탄핵은 국회의 탄핵소추와 헌법재판소의 탄핵심판을 거쳐야 하며, 이는 정치적 상황과 여론에 따라 달라질 수 있습니다. 따라서, 현재로서는 탄핵 여부에 대한 정확한 정보를 제공할 수 없습니다.
